In [ ]:
import os
import pandas as pd

# CSV 파일이 들어있는 폴더 경로
folder_path = '../../data'

# 불러올 CSV 파일 이름
csv_filename = 'data.csv'

# 전체 경로 생성
csv_path = os.path.join(folder_path, csv_filename)

# pandas로 CSV를 DataFrame으로 읽어들이기
df = pd.read_csv(csv_path, encoding="utf-8")

In [ ]:
# 2. 날짜 컬럼 형변환
df['조회일'] = pd.to_datetime(df['조회일'], errors='coerce')
df['개봉일'] = pd.to_datetime(df['개봉일'], errors='coerce')

# 3. 영화별 첫 조회일 추출
first_view_date = df.groupby('영화명')['조회일'].min().reset_index()
first_view_date.columns = ['영화명', '첫조회일']

# 4. 결측 개봉일을 영화별 첫 조회일로 대체
df = df.merge(first_view_date, on='영화명', how='left')
df['개봉일'] = df['개봉일'].fillna(df['첫조회일'])
df.drop(columns=['첫조회일'], inplace=True)

# 5. 상영일(개봉일로부터 며칠 지났는지) 계산
df['상영일'] = (df['조회일'] - df['개봉일']).dt.days

# 6. 영화명과 상영일 기준 내림차순 정렬
df_sorted = df.sort_values(['영화명', '상영일'], ascending=[True, False])

# 7. 결과 저장
df_sorted.to_csv("sorted_movie_data.csv", index=False, encoding='utf-8-sig')
print("✅ 정렬된 결과가 'sorted_movie_data.csv'로 저장되었습니다.")
